In [ ]:
# Install keras-nlp
!pip install -q keras-nlp tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.5/337.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 62.4 MB/s eta 0:00:00


In [ ]:
# Import librairies
import tensorflow as tf 
import numpy as np 
import pandas as pd
import keras_nlp
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
# Import dataset
dataset = pd.read_csv('/content/spam.csv', encoding="ISO-8859-1")

In [ ]:
# Rename columns to make apparent target and email
dataset = dataset.rename(columns={'v1':'Target', 'v2':'email'})

In [ ]:
# Delete empty columns
del dataset['Unnamed: 2']
del dataset['Unnamed: 3']
del dataset['Unnamed: 4']

In [ ]:
# Use a binary to determine if email is spam or ham. This column will be the target used by the model.
dataset['target_binary'] = dataset['Target'].apply(lambda x: 1.0 if x == 'spam' else 0.0)

In [ ]:
# Import preprocessor and encoder from the keras hub
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2')

In [ ]:
# Do train test split
xtrain, xval, ytrain, yval = train_test_split(dataset.email, dataset.target_binary, test_size=0.3, stratify = dataset.target_binary)

In [ ]:
# Build deep learning model
# Input layer to feed the data to the model
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
# Preprocessing layer
preprocessed_text = bert_preprocessor(text_input)
# Embedding layer
embeed = bert_encoder(preprocessed_text)
# Dropout unecessary neurons
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
# Output layer
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
# creat final model
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [ ]:
# check the summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['Inputs[0][0]']                 
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
# Define metrics of model. SInce it is a binary classification problem, the accuracy is binary.
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]

In [ ]:
# Compile our model
model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [ ]:
# Fit model
history = model.fit(xtrain, ytrain, epochs = 10)

Epoch 1/10
122/122 [==============================] - 25s 166ms/step - loss: 0.2852 - accuracy: 0.8792 - precision: 0.8714 - recall: 0.1166
Epoch 2/10
122/122 [==============================] - 19s 156ms/step - loss: 0.2022 - accuracy: 0.9236 - precision: 0.9061 - recall: 0.4799
Epoch 3/10
122/122 [==============================] - 20s 165ms/step - loss: 0.1723 - accuracy: 0.9382 - precision: 0.8832 - recall: 0.6214
Epoch 4/10
122/122 [==============================] - 20s 165ms/step - loss: 0.1547 - accuracy: 0.9462 - precision: 0.8826 - recall: 0.6902
Epoch 5/10
122/122 [==============================] - 20s 167ms/step - loss: 0.1447 - accuracy: 0.9503 - precision: 0.8889 - recall: 0.7189
Epoch 6/10
122/122 [==============================] - 19s 158ms/step - loss: 0.1361 - accuracy: 0.9536 - precision: 0.8869 - recall: 0.7495
Epoch 7/10
122/122 [==============================] - 20s 167ms/step - loss: 0.1285 - accuracy: 0.9559 - precision: 0.8944 - recall: 0.7610
Epoch 8/10
122/122 [

In [ ]:
# Evaluate performance
model.evaluate(xval,yval)

53/53 [==============================] - 10s 172ms/step - loss: 0.1157 - accuracy: 0.9635 - precision: 0.9055 - recall: 0.8125


[0.11571221798658371, 0.9635167717933655, 0.9054726362228394, 0.8125]